In [1]:
# Импорт всех необходимых библиотек
!pip install catboost xgboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostClassifier
import xgboost as xgb

data = pd.read_excel('Данные_для_курсовои_Классическое_МО.xlsx').drop(columns=['Unnamed: 0'])
data.dropna(inplace=True)

# Переместим названия колонок в отдельную переменную, пригодится
columns = data.columns.tolist()

# Быстрая проверка
data.head()

#'IC50, mM', 'CC50, mM', 'SI'

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# Создаём бинарную целевую переменную
ic50_median = data['IC50, mM'].median()
print(f"Медиана IC50: {ic50_median}")

y = (data['IC50, mM'] > ic50_median).astype(int)
X = data.drop(columns=['IC50, mM', 'CC50, mM', 'SI'])

# Разбиение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Медиана IC50: 45.99200608501366


Логистическая регрессия
---

In [3]:
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train)
y_pred_logreg = logreg.predict(X_test_scaled)

print("Логистическая регрессия:")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("F1:", f1_score(y_test, y_pred_logreg))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_logreg))

Логистическая регрессия:
Accuracy: 0.7
F1: 0.7142857142857143
ROC-AUC: 0.7


In [4]:
rf_model = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("Random Forest:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("F1:", f1_score(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_rf))

Random Forest:
Accuracy: 0.745
F1: 0.7582938388625592
ROC-AUC: 0.745


In [5]:
cat_model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    random_seed=42,
    verbose=0
)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)

print("CatBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_cat))
print("F1:", f1_score(y_test, y_pred_cat))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_cat))

CatBoost:
Accuracy: 0.72
F1: 0.7307692307692307
ROC-AUC: 0.72


In [6]:
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=5,
    learning_rate=0.05,
    random_state=42
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost:")
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("F1:", f1_score(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, y_pred_xgb))

XGBoost:
Accuracy: 0.705
F1: 0.7121951219512195
ROC-AUC: 0.705


In [7]:
results_ic50_clf = pd.DataFrame({
    'Модель': ['Logistic Regression', 'Random Forest', 'CatBoost', 'XGBoost'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_logreg),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_cat),
        accuracy_score(y_test, y_pred_xgb)
    ],
    'F1': [
        f1_score(y_test, y_pred_logreg),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_cat),
        f1_score(y_test, y_pred_xgb)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_pred_logreg),
        roc_auc_score(y_test, y_pred_rf),
        roc_auc_score(y_test, y_pred_cat),
        roc_auc_score(y_test, y_pred_xgb)
    ]
})

display(results_ic50_clf)

,Модель,Accuracy,F1,ROC-AUC
0,Logistic Regression,0.700,0.714286,0.700
1,Random Forest,0.745,0.758294,0.745
2,CatBoost,0.720,0.730769,0.720
3,XGBoost,0.705,0.712195,0.705


---

Logistic Regression
Это простая базовая модель — даёт 70% точности.

Видно, что в этой задаче есть нелинейные зависимости, которые логистическая модель уже не может полностью уловить.

Это наша средняя точка.

---

Random Forest (лучшая модель на данный момент)
Дал Accuracy 74.5% и F1 — 75.8%.

За счёт того, что Random Forest может ловить сложные нелинейные зависимости между признаками, он показывает уверенно лучший результат.

Очень сбалансирован между точностью и полнотой (что видно по F1-score).

---

CatBoost
Чуть слабее Random Forest, но достаточно стабильно — 72% точности.

Это всё ещё хороший результат, но видимо, здесь Random Forest оказался немного более удачным.

---

XGBoost
Показывает схожий результат с CatBoost, но немного слабее — 70.5%.

Видимо, в наших данных XGBoost не получил такого преимущества, которое он иногда даёт на более сложных выборках.

---

Итоговые выводы:
Задача решаема — мы можем классифицировать, находится ли вещество "выше/ниже" медианы с точностью 74-75%.

Лучший результат даёт смесь медианы и Random Forest.

Сложные бустинги (CatBoost, XGBoost) здесь не дали ощутимого прироста.

Простая логистическая регрессия показала, что проблема не линейная, но и не слишком сложная — около 70% точности уже в базовой модели.